In [15]:
!pip install wandb

In [16]:
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms.transforms import Resize
from torchvision import transforms
from tqdm import tqdm

In [18]:
wandb.init(project="persian-mnist_transferLearning")

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torchvision.models.resnet101(pretrained=True)

In [20]:
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 10)

In [21]:
ct = 0
for child in model.children():
    ct += 1
    if ct < 8:
        for param in child.parameters():
            param.requires_grad = False

HYPER PARAMETER

In [22]:
config = wandb.config
config.learning_rate = 0.001
config.batch_size = 64
config.epochs = 20

In [23]:
model = model.to(device)

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
transform = transforms.Compose([
                              transforms.RandomRotation(10),
                              transforms.Resize((32,32)),
                              transforms.ToTensor(),
                              transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
dataset = torchvision.datasets.ImageFolder(root="/content/drive/MyDrive/MNIST_persian", transform = transform)
train_data = torch.utils.data.DataLoader(dataset, batch_size=config.batch_size, shuffle=True, num_workers=2)

COMPILE

In [31]:
optimizer=torch.optim.Adam(model.parameters(),lr=config.learning_rate)
loss_function=nn.CrossEntropyLoss()

def calc_acc(preds, labels):
  _, pred_max = torch.max(preds, 1)
  acc = torch.sum(pred_max == labels.data, dtype=torch.float64) / len(preds)
  return acc

In [32]:
wandb.watch(model)
for epoch in range(config.epochs):
  train_loss = 0.0
  train_acc = 0.0

  for images, labels in tqdm(train_data):
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()
    preds = model(images)

    loss = loss_function(preds, labels)
    loss.backward()

    optimizer.step()

    train_loss += loss
    train_acc += calc_acc(preds, labels)

  total_loss = train_loss / len(train_data)
  total_acc = train_acc / len(train_data)
  if epoch % 2 == 0:
    wandb.log({"loss": total_loss})
    wandb.log({"acc": total_acc})


  print(f"Epoch: {epoch}, Loss: {total_loss}, Accuracy: {total_acc}")


100%|██████████| 19/19 [01:39<00:00,  5.22s/it]


Epoch: 0, Loss: 1.638708233833313, Accuracy: 0.4503837719298245


100%|██████████| 19/19 [00:03<00:00,  5.94it/s]


Epoch: 1, Loss: 0.8591595888137817, Accuracy: 0.7025767543859649


100%|██████████| 19/19 [00:03<00:00,  5.85it/s]


...

Epoch: 17, Loss: 0.19521982967853546, Accuracy: 0.9358552631578947


100%|██████████| 19/19 [00:03<00:00,  6.02it/s]


Epoch: 18, Loss: 0.1626596748828888, Accuracy: 0.9468201754385964


100%|██████████| 19/19 [00:03<00:00,  6.01it/s]

Epoch: 19, Loss: 0.14854374527931213, Accuracy: 0.9501096491228069


In [ ]:
torch.save(model.state_dict(), "mnist_persian_transfer_learning.pth")